# A. State preparation with Jaynes-Cummings controls

In [ ]:
# ruff: noqa
from feedback_grape.fgrape import optimize_pulse_parameterized
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [3]:
N_cav = 4

In [4]:
def qubit_unitary(alpha):
    return jnp.exp(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conj() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [5]:
def qubit_cavity_unitary(beta):
    return jnp.exp(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conj()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
            / 2
        )
    )

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [6]:
time_steps = 3  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [7]:
psi0 = tensor(basis(2), basis(N_cav))
psi_target = tensor(basis(2), fock(N_cav, 1) + fock(N_cav, 3))
psi_target = psi_target / jnp.linalg.norm(psi_target)

In [ ]:
print(fock(N_cav,1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

In [8]:
parameters = jnp.array([jnp.pi / 2, 0.0])
result_shapes = [
    gate(param).shape
    for gate, param in zip([qubit_unitary, qubit_cavity_unitary], parameters)
]
print(f"Shapes of results: {result_shapes}")

Shapes of results: [(8, 8), (8, 8)]


In [9]:
import jax.random as random
key = random.PRNGKey(0)  # Random seed for reproducibility
num_gates = len([qubit_unitary, qubit_cavity_unitary])  # Number of parameterized gates
initial_parameters = random.uniform(
    key, shape=(time_steps, num_gates), minval=-2 * jnp.pi, maxval=2 * jnp.pi
)
print(initial_parameters)

[[-1.02469816 -3.56513646]
 [ 5.84740194  0.93620132]
 [ 0.40496999 -1.82331528]]


In [10]:
result = optimize_pulse_parameterized(
    U_0=psi0,
    C_target=psi_target,
    feedback=False,
    parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
    initial_parameters=initial_parameters,
    mode="nn",
    goal="fidelity",
    num_time_steps=time_steps,
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    type="state",
    propcomp="time-efficient",
    learning_rate=0.0001,
)

In [11]:
print(result)

result(control_amplitudes=Array([[-1.02449888, -3.56493885],
       [ 5.84760186,  0.93640128],
       [ 0.40516999, -1.82311528]], dtype=float64), final_fidelity=Array(0.25053114, dtype=float64), iterations=2, final_operator=Array([[17523.43977517+2489.09857761j],
       [17584.2665321 +2569.94272629j],
       [17573.72195233+2511.64355286j],
       [17555.86847795+2609.23610445j],
       [17584.17649092+2533.43991479j],
       [17526.72112003+2614.24435363j],
       [17586.85493072+2551.99405924j],
       [17523.43977517+2489.09857761j]], dtype=complex128))
